# Download packages

In [1]:
%pip install tensorly

Note: you may need to restart the kernel to use updated packages.


# Import libraries

In [2]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import mode_dot
from typing import Union

In [3]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import mode_dot

In [4]:
# Устанавливаем бэкэнд TensorLy на NumPy (по умолчанию)
tl.set_backend('numpy')

# Algoritm using article idea

In [5]:
def tensor_based_compressive_sensing(
    A: Union[np.ndarray, tl.tensor],
    P: Union[np.ndarray, tl.tensor],
    Y: Union[np.ndarray, tl.tensor],
    max_iter: int,
    epsilon: float,
    lambd: float,
    delta_0: float,
    delta_max: float
) -> tl.tensor:
    """
    Implements the Tensor-based Compressive Sensing Algorithm (Algorithm 3).
    
    Parameters:
    - A: Input tensor of shape (I x J x W) (numpy array or Tensorly tensor)
    - P: Permutation matrix or sensor selection matrix (should be compatible for mode-0 multiplication with A)
         (numpy array or Tensorly tensor)
    - Y: Observed data tensor (after sensor placement) (numpy array or Tensorly tensor)
    - max_iter: Maximum number of iterations (int)
    - epsilon: Small positive scalar for shrinkage thresholding (float)
    - lambd: Regularization parameter (float)
    - delta_0: Initial value for the Lagrange penalty term (float)
    - delta_max: Maximum value for delta (float)

    Returns:
    - x_hat: Recovered sparse code vector of shape (W x 1) (Tensorly tensor)
    """

    # Ensure A, P, Y are tensors
    A = tl.tensor(A, dtype=tl.float32)
    P = tl.tensor(P, dtype=tl.float32)
    Y = tl.tensor(Y, dtype=tl.float32)

    # Get dimensions
    I, J, W = A.shape

    # Initialize variables
    x_n = tl.zeros((W, 1), dtype=tl.float32)
    d_n = tl.zeros((W, 1), dtype=tl.float32)
    p_n = tl.zeros((W, 1), dtype=tl.float32)
    delta_n = delta_0

    # Preprocess A: A = P x1 A (since P is (N x I), we use mode-0 multiplication)
    A = mode_dot(A, P, mode=0)  # A now has shape (N, J, W)

    # Unfold A along mode-2 (mode-3 in MATLAB notation)
    A_unfold = tl.unfold(A, mode=2)  # Shape: (W, N * J)

    # Compute A_T_A and A_Y
    A_T_A = A_unfold @ A_unfold.T  # Shape: (W, W)

    # Reshape Y to match the dimensions
    Y_vector = Y.reshape(-1, 1)  # Shape: (N * J, 1)

    # Compute A_Y
    A_Y = A_unfold @ Y_vector  # Shape: (W, 1)

    # Start iterations
    for n in range(1, max_iter + 1):
        # Update x_n
        # Solve: (A_T_A + delta_n * I) * x_n = A_Y + delta_n * (d_n - p_n)
        left_matrix = A_T_A + delta_n * np.eye(W)
        right_vector = A_Y + delta_n * (d_n - p_n)
        x_n = np.linalg.solve(left_matrix, right_vector)

        # Compute x_hat
        x_hat = lambd * x_n + (1 - lambd) * d_n

        # Update d_n using iterative shrinkage-thresholding
        threshold = epsilon / delta_n
        temp = x_hat + p_n
        d_n = np.maximum(0, temp - threshold) - np.maximum(0, -temp - threshold)

        # Update p_n
        p_n = p_n + x_hat - d_n

        # Update delta_n
        delta_n = min(delta_n, delta_max)

    return x_hat

In [9]:
# Example dimensions
I, J, W = 5, 5, 10  # Adjust as needed
N = 3  # Number of sensors (should be less than or equal to I)

# Create random tensors for A, P, and Y
A = tl.tensor(np.random.randn(I, J, W), dtype=tl.float32)

# Create a random permutation/sensor selection matrix P (N x I)
indices = np.random.choice(I, N, replace=False)
P = np.eye(I)[indices]  # Shape: (N, I)

# Generate observed data Y (N x J)
Y = mode_dot(A, P, mode=0)
Y = Y.mean(axis=2)  # Aggregate along the W dimension for simplicity

# Set algorithm parameters
max_iter = 50
epsilon = 1e-3
lambd = 0.1
delta_0 = 1.0
delta_max = 10.0

# Call the algorithm
x_hat = tensor_based_compressive_sensing(A, P, Y, max_iter, epsilon, lambd, delta_0, delta_max)

# x_hat contains the recovered sparse code vector
print("Recovered x_hat:")
print(x_hat)

Recovered x_hat:
[[0.09589178]
 [0.09768766]
 [0.0992037 ]
 [0.09915997]
 [0.10180801]
 [0.09708412]
 [0.09756022]
 [0.09633015]
 [0.09829935]
 [0.10043628]]
